In [1]:
import numpy as np
import pandas as pd
from text_proc import text_process
import json

[nltk_data] Downloading package stopwords to /tmp...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /tmp...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to /tmp...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /tmp...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
!pip install nltk

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
with open('IMDB_reviews.json') as json_file:      
    data = json_file.readlines()
    data = list(map(json.loads, data)) 
    
df = pd.DataFrame(data)

In [4]:
df.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [5]:
df.shape

(573913, 7)

In [6]:
req_df = df[['is_spoiler','review_text']]

In [7]:
req_df.head()

,is_spoiler,review_text
0,True,"In its Oscar year, Shawshank Redemption (writt..."
1,True,The Shawshank Redemption is without a doubt on...
2,True,I believe that this film is the best story eve...
3,True,"**Yes, there are SPOILERS here**This film has ..."
4,True,At the heart of this extraordinary movie is a ...


In [8]:
req_df.columns = ['label','text']

In [9]:
req_df.head()

,label,text
0,True,"In its Oscar year, Shawshank Redemption (writt..."
1,True,The Shawshank Redemption is without a doubt on...
2,True,I believe that this film is the best story eve...
3,True,"**Yes, there are SPOILERS here**This film has ..."
4,True,At the heart of this extraordinary movie is a ...


# Converting the categorical labels to numerical equivalent

In [10]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
req_df['label'] = label_enc.fit_transform(req_df['label'])
req_df.head()

c:\users\what's in\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,label,text
0,1,"In its Oscar year, Shawshank Redemption (writt..."
1,1,The Shawshank Redemption is without a doubt on...
2,1,I believe that this film is the best story eve...
3,1,"**Yes, there are SPOILERS here**This film has ..."
4,1,At the heart of this extraordinary movie is a ...


In [11]:
req_df = req_df.sample(frac=1).reset_index(drop=True)
req_df.head()

,label,text
0,0,"When this film came out, I ,again, expected an..."
1,0,"May not be JUST as good as the original, but d..."
2,0,I am someone who does not especially like coll...
3,0,This is a case where the movie is just awful. ...
4,0,On the tail end of a modest musical revival be...


In [12]:
req_df.groupby('label').describe()

text                                                                
        count  unique                                                top freq
label                                                                        
0      422989  422601  I have never seen such an amazing film since I...   13
1      150924  150862  OK, I finally got my chance to see The Dark Kn...    4

In [13]:
orig_df = req_df.drop_duplicates(subset='text').reset_index(drop=True)

In [14]:
orig_df.groupby('label').describe()

text                                                                
        count  unique                                                top freq
label                                                                        
0      422568  422568  The suspense kept taking me down into the dram...    1
1      150817  150817  This one of the greatest sequels of all time, ...    1

# Taking only 25000 records from each label

In [15]:
label_0 = req_df.loc[req_df['label'].isin([0])].groupby('label').head(25000)
label_0.head()
label_1 = req_df.loc[req_df['label'].isin([1])].groupby('label').head(25000)
label_1.head()

,label,text
8,1,"I dislike gratuitous violence, loud screeching..."
17,1,"It is very big, and full of pretty American mo..."
26,1,Basically the director had some very cool idea...
36,1,Triangle is an surrealistic movie which was as...
37,1,When i first saw the leg of one of the girl in...


In [16]:
df_new = pd.concat([label_0,label_1])
df_new.head()

,label,text
0,0,"When this film came out, I ,again, expected an..."
1,0,"May not be JUST as good as the original, but d..."
2,0,I am someone who does not especially like coll...
3,0,This is a case where the movie is just awful. ...
4,0,On the tail end of a modest musical revival be...


In [17]:
df_new['text'][3]

'This is a case where the movie is just awful. Nothing good can be said by a sensible intelligent person about this film, period. That said, what the 10 star young people seem to rave about is the "experience" part of the screening. As "Rocky Horror" did years ago, "The Room" has created a cult following of young people (mostly) who go to the film to make fun of the corny dialogue and silly premises. As with any "experience" film, the fun is that most of the audience has already seen the movie. They shout out the dialogue and give appropriate groans when the dialogue is exceptionally bad (which is almost always the case). If that is what you crave - a social interactive film experience - then "The Room" is your kind of movie. It has nothing else to offer... really. It is not an independent classic. It is not the great insight of a film genius. It is not the latest trend in cinema (although it closely resembles aspects of reality television). And, it is not a good movie. It is a very, v

# Shuffling the dataset as it is ordered by each label

In [18]:
df_new = df_new.sample(frac=1).reset_index(drop=True)
df_new.head()

,label,text
0,1,I got this at the bargain bin at a used DVD st...
1,0,I rented this movie this last weekend becouse ...
2,1,Judging a 1993 movie in 2016 &Especially that'...
3,0,A little Indian boy (Sunny Pawar) gets separat...
4,0,Audrey Tautou has a great screen presence and ...


In [19]:
df_new.to_csv('Intermediate_data.csv',index=False)

In [17]:
df_new.groupby('label').describe()

text                                                               
       count unique                                                top freq
label                                                                      
0      25000  24995  The ambitious Eve Harrington (Anne Baxter) get...    2
1      25000  24999  Christopher Nolan's second bundle of joy "The ...    2

In [18]:
df_new['length']= df_new['text'].apply(len)
df_new.head()

,label,text,length
0,0,"When walking out of the theater from ""Tron Leg...",3120
1,1,"Okey, i had actually pretty high expectations ...",1506
2,0,Spirited Away is pure magic from beginning to ...,761
3,1,"Tony Wendice (Ray Milland), a former English p...",1024
4,0,"I had the chance to see ""The Princess Diaries""...",772


In [19]:
df_new.length.describe()

count    50000.000000
mean      1589.072640
std       1187.776282
min         32.000000
25%        764.000000
50%       1164.000000
75%       2018.000000
max      14081.000000
Name: length, dtype: float64

In [146]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_process).fit(df_new['text'])
print(len(bow_transformer.vocabulary_))

36971


In [148]:
bow4 = bow_transformer.transform(df_new['text'])

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
pipeline = Pipeline([
                    ('bow', CountVectorizer(analyzer=text_process)),
                    ('tfidf',TfidfTransformer()),
                    ('classifier', MultinomialNB())
])

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df_new['text'],df_new['label'],test_size=0.2)

In [23]:
x_test.head()

13783    This was a discussion not a movie. I would sum...
46008    When talking to friends and family about this ...
3239     SERIOUSLY MAJOR SPOILERS CONCERNING PRETTY MUC...
17460    OK, so I saw it in cinemas after being really ...
16538    let me tell you how much i hate this movie : t...
Name: text, dtype: object

In [24]:
print(len(x_train),len(x_test),len(y_train),len(y_test))

40000 10000 40000 10000


In [25]:
model = pipeline.fit(x_train,y_train)

In [26]:
y_pred = model.predict(x_test)

In [30]:
y_pred1 = model.predict(["movie was good"])

In [36]:
out = ['spoiler' if i else 'not spoiler' for i in y_pred1]
out
for i in y_pred1:
    if i:
        text = 'spoiler'
    else:
        text = 'not spoiler'
        
print(text)

spoiler


In [27]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.37      0.50      5028
           1       0.58      0.89      0.71      4972

    accuracy                           0.63     10000
   macro avg       0.68      0.63      0.60     10000
weighted avg       0.68      0.63      0.60     10000



In [28]:
import joblib
joblib.dump(model,'NB1.joblib')

['NB1.joblib']

In [1]:
import pickle

In [ ]:
pickle.dump(model,open('model.pkl','wb'))

In [3]:
remodel = pickle.load(open('model.pkl','rb'))

In [4]:
y_pred2 = remodel.predict(["The movies was good!", "The movie was predictable and in the climax, alien visits earth"])

In [5]:
out = ['Spoiler' if i else 'No Spoiler' for i in y_pred2]
out

['No Spoiler', 'Spoiler']

In [35]:
reconst_model = joblib.load('NB1.joblib')

In [45]:
#the movie could have been better. The director's work was visible in screenplay and the villain was spectacular"
y_pred1 = reconst_model.predict(["The director's vision was great","the movie could have been better. The director's work was visible in screenplay"])

In [46]:
out1 = ['Spoiler' if i else 'No Spoiler' for i in y_pred1]
out1

['No Spoiler', 'No Spoiler']